In [95]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [96]:
# imports
import torch
from pytorch3d.transforms.so3 import (
    so3_exp_map,
    so3_relative_angle,
)
from pytorch3d.renderer.cameras import (
    PerspectiveCameras,
)

# add path for demo utils
import sys
import os
sys.path.append(os.path.abspath(''))

# set for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [97]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/camera_visualization.py
from camera_visualization import plot_camera_scene

!mkdir data
!wget -P data https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/data/camera_graph.pth

--2022-09-29 18:22:34--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/camera_visualization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: 'camera_visualization.py.1'

camera_visualizatio 100%[===================>]   1.98K  --.-KB/s    in 0s      

2022-09-29 18:22:35 (17.4 MB/s) - 'camera_visualization.py.1' saved [2029/2029]

mkdir: data: File exists
--2022-09-29 18:22:35--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/data/camera_graph.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.13

In [98]:
## Setup Camera
camera_graph_file = './data/camera_graph.pth'
(R_absolute_gt, T_absolte_gt), (R_relative, T_relative), relative_edges = torch.load(camera_graph_file)

cameras_relative = PerspectiveCameras(
    R = R_relative.to(device),
    T = R_relative.to(device),
    device = device
)

cameras_absolute_gt = PerspectiveCameras(
    R = R_absolute_gt.to(device),
    T = T_absolte_gt.to(device),
    device = device
)

N = R_absolute_gt.shape[0]

In [99]:
def calc_camera_distance(cam_1, cam_2):

    R_distance = (1.-so3_relative_angle(cam_1.R, cam_2.R, cos_angle=True)).mean()

    T_distance = ((cam_1.T - cam_2.T)**2).sum(1).mean()

    return R_distance + T_distance

def get_relative_camera(cams, edges):
    trans_i, trans_j = [
        PerspectiveCameras(
            R = cams.R[edges[:, i]],
            T = cams.T[edges[:, i]],
            device = device,
        ).get_world_to_view_transform()
        for i in (0, 1)
    ]

    trans_rel = trans_i.inverse().compose(trans_j)

    matrix_rel = trans_rel.get_matrix()
    cams_relative = PerspectiveCameras(
        R = matrix_rel[:, :3, :3],
        T = matrix_rel[:, 3, :3],
        device = device,
    )

    return cams_relative

In [100]:
# 초기화
log_R_absolute_init = torch.randn(N, 3, dtype = torch.float32, device = device)
T_absolute_init = torch.randn(N, 3, dtype = torch.float32, device = device)

log_R_absolute_init[0, :] = 0.0
T_absolute_init[0, :] = 0.0

log_R_absolute = log_R_absolute_init.clone().detach()
log_R_absolute.requires_grad = True

T_absolute = T_absolute_init.clone().detach()
T_absolute.requires_grad = True


camera_mask = torch.ones(N, 1, dtype = torch.float32, device = device)
print(camera_mask[0])
camera_mask[0] = 0.0


optimizer = torch.optim.SGD([ log_R_absolute, T_absolute ], lr = .1, momentum = .9)


n_iter = 2000
loop = range(n_iter)
for i in range(n_iter):
    optimizer.zero_grad() # 최적화 경사도 초기화
    R_absolute = so3_exp_map(log_R_absolute * camera_mask)

    ##
    cameras_absolute = PerspectiveCameras(
        R = R_absolute,
        T = T_absolute * camera_mask,
        device = device,
    )

    ##
    cameras_relative_composed = get_relative_camera(cameras_absolute, relative_edges)
    camera_distance = calc_camera_distance(cameras_relative_composed, cameras_relative)

    ##
    camera_distance.backword()

    ##
    optimizer.step()

    ## 반복문이 끝나는
    if i % 200==0 or i==n_iter-1:
        status = 'iteration=%3d; camera_distance=%1.3e' % (i, camera_distance)
        plot_camera_scene(cameras_absolute, cameras_absolute_gt, status)


print('Optimization finished.')

tensor([1.])


RuntimeError: The size of tensor a (190) must match the size of tensor b (3) at non-singleton dimension 1